# Vanna Ai Exploration 


## Generate dataframe with random data

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import sqlite3
import os
#pip install vanna
# import vanna
# from vanna.remote import VannaDefault
# Vanna_API_Key = os.getenv('Vanna_API_Key')

In [3]:
#print(f"The value of 'MY_VARIABLE' is: {Vanna_API_Key}")

In [4]:
# Generate random patient IDs
patient_ids = [f'P{str(i).zfill(4)}' for i in range(1, 101)]

# Generate random service dates within the past two weeks
today = datetime.today()
service_dates = [today - timedelta(days=random.randint(0, 14)) for _ in range(100)]

# Sample list of chronic ICD-10 codes and their descriptions
icd_10_codes = [
    ('E11', 'Type 2 diabetes mellitus'),
    ('I10', 'Essential (primary) hypertension'),
    ('J45', 'Asthma'),
    ('M54', 'Dorsalgia (back pain)'),
    ('K50', 'Crohn’s disease'),
    ('K51', 'Ulcerative colitis')
]

# Generate random ICD-10 codes and descriptions
icd_10_data = [random.choice(icd_10_codes) for _ in range(100)]
icd_10_codes, icd_10_descriptions = zip(*icd_10_data)

# Create the DataFrame
df = pd.DataFrame({
    'patient_id': patient_ids,
    'service_date': service_dates,
    'icd_10_code': icd_10_codes,
    'icd_10_description': icd_10_descriptions
})

# Display the first few rows of the DataFrame
df.head()

,patient_id,service_date,icd_10_code,icd_10_description
0,P0001,2025-01-04 15:00:19.823255,I10,Essential (primary) hypertension
1,P0002,2025-01-09 15:00:19.823255,E11,Type 2 diabetes mellitus
2,P0003,2025-01-09 15:00:19.823255,M54,Dorsalgia (back pain)
3,P0004,2025-01-04 15:00:19.823255,E11,Type 2 diabetes mellitus
4,P0005,2024-12-28 15:00:19.823255,K51,Ulcerative colitis


# Load data to sqlite db

In [13]:

# Connect to a SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('chronic_disease_example.db')

# Save the DataFrame to the SQLite database
df.to_sql('patients', conn, if_exists='replace', index=False)

# Function to query the SQLite database using SQL
def query_dataframe(sql_query):
    return pd.read_sql_query(sql_query, conn)

# Example usage
sql_query = """
SELECT icd_10_description, COUNT(*) as number_of_patients 
FROM patients 
GROUP BY icd_10_description
--select * from patients
"""
result = query_dataframe(sql_query)
print(result)

# Close the connection
conn.close()


                 icd_10_description  number_of_patients
0                            Asthma                  16
1                   Crohn’s disease                  18
2             Dorsalgia (back pain)                  18
3  Essential (primary) hypertension                  15
4          Type 2 diabetes mellitus                  17
5                Ulcerative colitis                  16


# Use Vanna Ai

In [ ]:
#https://vanna.ai/docs/sqlite-openai-vanna-vannadb/

Convert the DataFrame to a SQL table using SQLite and define a function query_dataframe that takes a plain English query,
 converts it to SQL using Vanna AI, and returns the results.

You can input any query in plain English,
 and the function will convert it to SQL and return the results.
 Make sure to replace query with your desired question.

In [19]:
import vanna
from vanna.remote import VannaDefault
Vanna_API_Key = os.getenv('Vanna_API_Key')
vanna_model_name = "disease"
api_key=Vanna_API_Key
vn = VannaDefault(model=vanna_model_name, api_key=api_key)

In [20]:
vn.connect_to_sqlite('chronic_disease_example.db')

In [21]:
df_ddl = vn.run_sql("SELECT type, sql FROM sqlite_master WHERE sql is not null")

for ddl in df_ddl['sql'].to_list():
  vn.train(ddl=ddl)

Adding ddl: CREATE TABLE "patients" (
"patient_id" TEXT,
  "service_date" TIMESTAMP,
  "icd_10_code" TEXT,
  "icd_10_description" TEXT
)


In [24]:
# # The following are methods for adding training data. Make sure you modify the examples to match your database.

# # DDL statements are powerful because they specify table names, colume names, types, and potentially relationships
# vn.train(ddl="""
#     CREATE TABLE IF NOT EXISTS my-table (
#         id INT PRIMARY KEY,
#         name VARCHAR(100),
#         age INT
#     )
# """)

# # Sometimes you may want to add documentation about your business terminology or definitions.
vn.train(documentation="Our business provides insight into disease incidence and prevelance.")

# # You can also add SQL queries to your training data. This is useful if you have some queries already laying around. You can just copy and paste those from your editor to begin generating new SQL.
vn.train(sql="SELECT icd_10_description, COUNT(*) as number_of_patients FROM patients GROUP BY icd_10_description")

Adding documentation....
Question generated with sql: What are the number of patients for each unique ICD-10 description? 
Adding SQL...


'4ec6e375916e39979d3b8751d558d426-sql'

In [25]:
# At any time you can inspect what training data the package is able to reference
training_data = vn.get_training_data()
training_data

,id,training_data_type,question,content
0,457448-ddl,ddl,None,"CREATE TABLE ""patients"" (\n""patient_id"" TEXT,\..."
1,2819555-doc,documentation,None,Our business provides insight into disease inc...
2,866744-sql,sql,What are the number of patients for each uniqu...,"SELECT icd_10_description, COUNT(*) as number_..."


In [ ]:
# # You can remove training data if there's obsolete/incorrect information. 
# vn.remove_training_data(id='1-ddl')

# ```## Asking the AI
# Whenever you ask a new question, it will find the 10 most relevant pieces of training data and use it as part of the LLM prompt to generate the SQL.
# ```python
# vn.ask(question=...)

In [ ]:
from vanna.flask import VannaFlaskApp
app = VannaFlaskApp(vn)
app.run()

Your app is running at:
http://localhost:8084
 * Serving Flask app 'vanna.flask'
 * Debug mode: on
None
